In [ ]:
import cv2
import glob
import pandas as pd
import urllib
import urllib3
from sklearn.model_selection import train_test_split
import os
import requests
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_images= glob.glob(r"drive/MyDrive/test/**/*.jpeg")


In [ ]:
len(train_images)

624

In [ ]:
folders = os.listdir(r"drive/MyDrive/test")

In [ ]:
dic_images={}
for fold in  folders:
    dic_images[fold]=glob.glob(r"drive/MyDrive/test/{}/*.jpeg".format(fold))

In [ ]:
dic_labels={}
for i,j in enumerate(folders):
    dic_labels[j] = i

In [ ]:
X,y=[],[]
for i,j in  dic_images.items():
    for img in j:
        g = cv2.imread(img)
        c= cv2.resize(g,(224,224))
        X.append(c)
        y.append(dic_labels[i])

In [ ]:
X = np.asarray(X).astype('float32')
y= np.array(y)

In [ ]:
X_resize =X/255.0

In [ ]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D,AvgPool2D,GlobalAvgPool2D,BatchNormalization
from tensorflow.keras.models import Sequential,Model
from keras_tuner.tuners import RandomSearch 

In [ ]:
import tensorflow as tf

def build_model(hp):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
            filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
            if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
                x = tf.keras.layers.MaxPool2D(padding='same')(x)
            else:
                x = tf.keras.layers.AvgPool2D(padding='same')(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
    x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
    optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),  
                                        loss=keras.losses.BinaryCrossentropy(from_logits=False), 
                                          metrics=['accuracy'])
    return model

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(build_model,
                     objective='accuracy',
                     max_epochs=3,
                     hyperband_iterations=2,
                     directory=r"drive/MyDrive/")

INFO:tensorflow:Reloading Oracle from existing project drive/MyDrive/untitled_project/oracle.json


In [ ]:
tuner.search(X_resize,y)

Trial 13 Complete [00h 00m 25s]
accuracy: 0.8878205418586731

Best accuracy So Far: 0.9150640964508057
Total elapsed time: 00h 05m 30s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 192)     5376      
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 192)    768       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 224, 224, 192)     0         
                                                                 
 average_pooling2d (AverageP  (None, 112, 112, 192)    0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 192)     331968

In [ ]:
model.fit(X_resize,y,epochs=6, batch_size=None, verbose='auto', callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=2, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)

Epoch 3/6
20/20 [==============================] - 13s 574ms/step - loss: 0.2218 - accuracy: 0.8926
Epoch 4/6
20/20 [==============================] - 11s 573ms/step - loss: 0.1497 - accuracy: 0.9503
Epoch 5/6
20/20 [==============================] - 11s 573ms/step - loss: 0.2233 - accuracy: 0.9054
Epoch 6/6
20/20 [==============================] - 11s 571ms/step - loss: 0.1796 - accuracy: 0.9279


In [ ]:
test_images= glob.glob(r"drive/MyDrive/val/**/*.jpeg")

In [ ]:
dic_images={}
for fold in  folders:
    dic_images[fold]=glob.glob(r"drive/MyDrive/val/{}/*.jpeg".format(fold))

In [ ]:
dic_labels={}
for i,j in enumerate(folders):
    dic_labels[j] = i

In [ ]:
X_test,y_test=[],[]
for i,j in  dic_images.items():
    for img in j:
        g = cv2.imread(img)
        c= cv2.resize(g,(224,224))
        X_test.append(c)
        y_test.append(dic_labels[i])

In [ ]:
X_test = np.asarray(X_test).astype('float32')
y_test= np.array(y_test)

In [ ]:
X_resacled =X_test/255.0

In [ ]:
loss,acc= model.evaluate(x=X_resacled, y=y_test, batch_size=None, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, return_dict=False)

3/3 [==============================] - 1s 375ms/step - loss: 1.2836 - accuracy: 0.4242


In [ ]:
acc

0.42424243688583374